# Basic loading data to sql database from csv file

In [1]:
import pandas as pd
import sqlalchemy as sa
from utils import connection as conn

In [2]:
# get credentials for oltp schema
host, database, user, password = conn.mysql_params('oltp')

In [3]:
engine = sa.create_engine(f"mysql://{user}:{password}@{host}:3306/{database}")
conn = engine.connect()

### Read data in the csv file
Instructions to download datasets from the Kaggle's public API here -> https://www.kaggle.com/docs/api
Dataset: https://www.kaggle.com/datasets/arnavsmayan/amazon-prime-userbase-dataset


In [4]:
# read data for fact table
prime = pd.read_csv('amazon_prime_users_dataset/amazon_prime_users_modified.csv')
prime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   User ID                        2500 non-null   int64 
 1   Name                           2500 non-null   object
 2   Email Address                  2500 non-null   object
 3   Username                       2500 non-null   object
 4   Date of Birth                  2500 non-null   object
 5   Gender                         2500 non-null   object
 6   location.Index                 2500 non-null   int64 
 7   Membership Start Date          2500 non-null   object
 8   Membership End Date            2500 non-null   object
 9   subscription plan.1.Index      2500 non-null   int64 
 10  payment.Index                  2500 non-null   int64 
 11  renewal.Index                  2500 non-null   int64 
 12  usage.Index                    2500 non-null   int64 
 13  pur

In [ ]:
prime.head(5)

### Match columns of CSV files to the same as in the server

In [ ]:
prime.columns

In [5]:
new_columns = [
    'User_ID',
    'Name',
    'Email_Address',
    'Username',
    'Date_of_Birth',
    'Gender',
    'Location_Id',
    'Membership_Start_Date',
    'Membership_End_Date',
    'Subscription_Plan_Id',
    'Payment_Info_Id',
    'Renewal_Status_Id',
    'Usage_Frequency_Id',
    'Purchase_History_Id',
    'Favorite_Genres_Id',
    'Devices_Used_Id',
    'Engagement_Metrics_Id',
    'Feedback_Ratings',
    'Customer_Support_Interactions'
]

In [6]:
for old_column, new_column in zip(prime.columns, new_columns):
    prime.rename(columns={old_column:new_column}, inplace=True)

prime.columns

Index(['User_ID', 'Name', 'Email_Address', 'Username', 'Date_of_Birth',
       'Gender', 'Location_Id', 'Membership_Start_Date', 'Membership_End_Date',
       'Subscription_Plan_Id', 'Payment_Info_Id', 'Renewal_Status_Id',
       'Usage_Frequency_Id', 'Purchase_History_Id', 'Favorite_Genres_Id',
       'Devices_Used_Id', 'Engagement_Metrics_Id', 'Feedback_Ratings',
       'Customer_Support_Interactions'],
      dtype='object')

In [8]:
# copy data from df to fact table
prime.to_sql(name='prime_users', con=conn, if_exists='append', index=False)

2500

### Validate the new data in server

In [9]:
check = pd.read_sql('SELECT * FROM prime_users', con=conn)

In [10]:
check.head(5)

,User_ID,Name,Email_Address,Username,Date_of_Birth,Gender,Location_Id,Membership_Start_Date,Membership_End_Date,Subscription_Plan_Id,Payment_Info_Id,Renewal_Status_Id,Usage_Frequency_Id,Purchase_History_Id,Favorite_Genres_Id,Devices_Used_Id,Engagement_Metrics_Id,Feedback_Ratings,Customer_Support_Interactions
0,1,Ronald Murphy,williamholland@example.com,williamholland,1953-06-03,Male,1,2024-01-15,2025-01-14,1,1,1,1,1,1,1,1,36,3
1,2,Scott Allen,scott22@example.org,scott22,1978-07-08,Male,2,2024-01-07,2025-01-06,2,2,1,1,1,2,2,1,38,7
2,3,Jonathan Parrish,brooke16@example.org,brooke16,1994-12-06,Female,3,2024-04-13,2025-04-13,2,1,1,1,2,3,1,2,33,8
3,4,Megan Williams,elizabeth31@example.net,elizabeth31,1964-12-22,Female,4,2024-01-24,2025-01-23,2,3,2,1,1,1,1,3,33,7
4,5,Kathryn Brown,pattersonalexandra@example.org,pattersonalexandra,1961-06-04,Male,5,2024-02-14,2025-02-13,1,2,2,2,3,4,1,2,43,1
